#### ID 2000

```Write a query that returns binary description of rate type per loan_id. The results should have one row per loan_id and two columns: for fixed and variable type.```

In [ ]:
%%sql
SELECT loan_id,
       CASE WHEN rate_type = 'fixed' THEN 1 ELSE 0 END    AS fixed,
       CASE WHEN rate_type = 'variable' THEN 1 ELSE 0 END AS variable
FROM submissions;

In [ ]:
df = submissions
pd.get_dummies(df[['loan_id', 'rate_type']], prefix='', prefix_sep='')

#### ID 2001

```Write a query that returns the rate_type, loan_id, loan balance , and a column that shows with what percentage the loan's balance contributes to the total balance among the loans of the same rate type```

In [ ]:
%%sql
SELECT rate_type,
       loan_id,
       SUM(balance) OVER (PARTITION BY loan_id)                     AS balance,
       balance * 100.0 / SUM(balance) OVER (PARTITION BY rate_type) AS balance_share
FROM submissions;

In [ ]:
df = submissions
df.groupby('loan_id', as_index=False).agg(balance=('balance', 'sum'))
df = pd.merge(
    pd.merge(df, df.groupby('rate_type', as_index=False).agg(balance_by_type=('balance', 'sum')), on='rate_type').drop(
        columns='balance'), df.groupby('loan_id', as_index=False).agg(balance=('balance', 'sum')), on='loan_id')
df['balance_share'] = df['balance'] * 100 / df['balance_by_type']
df[['loan_id', 'rate_type', 'balance', 'balance_share']]

#### ID 2002

```Write a query that returns the user ID of all users that have created at least one ‘Refinance’ submission and at least one ‘InSchool’ submission.```

In [ ]:
%%sql
SELECT DISTINCT user_id
FROM loans
WHERE user_id IN (SELECT user_id FROM loans WHERE type IN ('Refinance'))
  AND user_id IN (SELECT user_id FROM loans WHERE type IN ('InSchool'))

In [ ]:
df_ref = loans.query('type == "Refinance"')['user_id'].drop_duplicates()
df_sch = loans.query('type == "InSchool"')['user_id'].drop_duplicates()
df = pd.merge(df_ref, df_sch, on='user_id')

#### ID 2003

```Write a query that joins this submissions table to the loans table and returns the total loan balance on each user’s most recent ‘Refinance’ submission. Return all users and the balance for each of them.```

In [ ]:
%%sql
WITH cte AS (SELECT user_id,
                    balance,
                    DENSE_RANK()
                    OVER (PARTITION BY user_id ORDER BY created_at DESC) AS rnk
             FROM loans l
                      JOIN submissions s ON l.id = s.loan_id
             WHERE type = 'Refinance')
SELECT user_id, balance
FROM cte
WHERE rnk = 1

In [ ]:
df = pd.merge(loans.query('type == "Refinance"'), submissions, how='inner', left_on='id', right_on='loan_id')
df['rnk'] = df.groupby('user_id')['created_at'].rank(method='first', ascending=False)
df.query('rnk == 1')[['user_id', 'balance']]

#### ID 2004

```Return the total number of comments received for each user in the 30 or less days before 2020-02-10. Don't output users who haven't received any comment in the defined time period.```

In [ ]:
%%sql
SELECT user_id,
       SUM(number_of_comments) AS number_of_comments
FROM fb_comments_count
WHERE ('2020-02-10' - created_at) BETWEEN 0 AND 30
GROUP BY user_id

In [ ]:
df = fb_comments_count
df[(pd.to_datetime('2020-02-10') - df['created_at']).dt.days.between(0, 30)].groupby('user_id', as_index=False).agg(
    n_comments=('number_of_comments', 'sum'))

#### ID 2005

```Output share of US users that are active. Active users are the ones with an "open" status in the table.```

In [ ]:
%%sql
SELECT COUNT(user_id) FILTER (WHERE status = 'open') * 1.0 / COUNT(user_id) *
       1.0 AS active_users_share
FROM fb_active_users
WHERE country = 'USA'

In [ ]:
df = fb_active_users
result = df.query('country == "USA" & status == "open"')['user_id'].nunique() / df.query('country == "USA"')[
    'user_id'].nunique()
result_df = pd.DataFrame({'index': ['user_count'], 'open': [result]})

#### ID 2006

```Return a distribution of users activity per day of the month. By distribution we mean the number of posts per day of the month.```

In [ ]:
%%sql
SELECT EXTRACT(DAY FROM post_date), COUNT(post_text)
FROM facebook_posts
GROUP BY EXTRACT(DAY FROM post_date)

In [ ]:
df = facebook_posts
df.groupby(df['post_date'].dt.day, as_index=False).agg(count=('post_text', 'count')).to_frame('user_activity')

#### ID 2007

```Which countries have risen in the rankings based on the number of comments between Dec 2019 vs Jan 2020? Hint: Avoid gaps between ranks when ranking countries!!!```

In [ ]:
%%sql
WITH dec_comments AS (SELECT country,
                             SUM(number_of_comments) AS total_comments,
                             DENSE_RANK() OVER (ORDER BY SUM(number_of_comments) DESC)
                                                     AS rnk
                      FROM fb_active_users AS au
                               JOIN fb_comments_count AS cc ON cc.user_id = au.user_id
                      WHERE EXTRACT(MONTH FROM created_at) = 12
                        AND EXTRACT(YEAR FROM created_at) = 2019
                      GROUP BY country),
     jan_comments AS (SELECT country,
                             SUM(number_of_comments) AS total_comments,
                             DENSE_RANK() OVER (ORDER BY SUM(number_of_comments) DESC)
                                                     AS rnk
                      FROM fb_active_users AS au
                               JOIN fb_comments_count AS cc ON cc.user_id = au.user_id
                      WHERE EXTRACT(MONTH FROM created_at) = 1
                        AND EXTRACT(YEAR FROM created_at) = 2020
                      GROUP BY country)
SELECT j.country
FROM jan_comments AS j
         FULL JOIN dec_comments AS d ON d.country = j.country
WHERE j.rnk < d.rnk
  OR d.rnk IS NULL;

In [ ]:
df_dec_comments = fb_comments_count.query('created_at.dt.year == 2019 & created_at.dt.month == 12').merge(
    fb_active_users, how='left', on='user_id').groupby('country', as_index=False).agg(
    total_comments=('number_of_comments', 'sum'))
df_dec_comments['rnk'] = df_dec_comments['total_comments'].rank(method='dense', ascending=False)

df_jan_comments = fb_comments_count.query('created_at.dt.year == 2020 & created_at.dt.month == 1').merge(
    fb_active_users, how='left', on='user_id').groupby('country', as_index=False).agg(
    total_comments=('number_of_comments', 'sum'))
df_jan_comments['rnk'] = df_jan_comments['total_comments'].rank(method='dense', ascending=False)

df = pd.merge(df_dec_comments, df_jan_comments, how='outer', on='country', suffixes=('_dec', '_jan')).query(
    'rnk_dec > rnk_jan | rnk_dec.isnull()')[['country']]

#### ID 2009

```Find users who are both a viewer and streamer.```

In [ ]:
%%sql
SELECT DISTINCT user_id
FROM twitch_sessions
WHERE user_id IN (SELECT user_id FROM twitch_sessions WHERE session_type = 'viewer')
  AND user_id IN (SELECT user_id FROM twitch_sessions WHERE session_type = 'streamer')

In [ ]:
df = pd.merge(twitch_sessions, twitch_sessions, how='inner', on='user_id', suffixes=('_user1', '_user2')).query(
    'session_type_user1 == "streamer" & session_type_user2 == "viewer"')[
    'user_id'].drop_duplicates().sort_values().reset_index(drop=True)

#### ID 2010

```List the top 10 users who accumulated the most sessions where they had more streaming sessions than viewing. Return the user_id, number of streaming sessions, and number of viewing sessions.```

In [ ]:
%%sql
WITH filtered AS (SELECT user_id,
                    COUNT(CASE WHEN session_type = 'streamer' THEN 1 ELSE NULL END) AS streaming_sessions,
                    COUNT(CASE WHEN session_type = 'viewer' THEN 1 ELSE NULL END)   AS viewing_sessions
             FROM twitch_sessions
             GROUP BY user_id
             HAVING COUNT(CASE WHEN session_type = 'streamer' THEN 1 ELSE NULL END) >
                    COUNT(CASE WHEN session_type = 'viewer' THEN 1 ELSE NULL END)),
     ranked AS (SELECT user_id,
                       streaming_sessions,
                       viewing_sessions,
                      DENSE_RANK()
                      OVER (ORDER BY (streaming_sessions + viewing_sessions) DESC) AS rnk
               FROM filtered)
SELECT user_id, streaming_sessions, viewing_sessions
FROM ranked
WHERE rnk <= 10

In [ ]:
# TODO

#### ID 2011

```Calculate the average session duration for each session type?```

In [ ]:
%%sql
SELECT session_type, AVG(session_end - session_start) AS duration
FROM twitch_sessions
GROUP BY session_type

In [ ]:
df = twitch_sessions
df['duration'] = df['session_end'] - df['session_start']
df.groupby('session_type', as_index=False).agg(duration=('duration', 'mean'))

#### ID 2012

```From users who had their first session as a viewer, how many streamer sessions have they had? Return the user id and number of sessions in descending order. In case there are users with the same number of sessions, order them by ascending user id.```

In [ ]:
%%sql
WITH ranked AS (SELECT user_id,
                       session_type,
                       DENSE_RANK()
                       OVER (PARTITION BY user_id ORDER BY session_start) AS rnk
                FROM twitch_sessions),
     first_viewer_users AS (SELECT user_id
                            FROM ranked
                            WHERE session_type = 'viewer'
                              AND rnk = 1)
SELECT user_id, COUNT(session_id) AS n_sessions
FROM twitch_sessions
WHERE user_id IN (SELECT user_id FROM first_viewer_users)
  AND session_type = 'streamer'
GROUP BY user_id
ORDER BY n_sessions DESC, user_id ASC

In [ ]:
df = twitch_sessions
df['rnk'] = df.groupby('user_id')['session_start'].rank(method='first', ascending=True)
first_viewer_users = df.query('session_type == "viewer" & rnk == 1')['user_id'].to_list()
df.query('session_type == "streamer" & user_id.isin(@first_viewer_users)').groupby('user_id', as_index=False).agg(
    n_sessions=('session_id', 'count')).sort_values(['n_sessions', 'user_id'], ascending=[False, True])

#### ID 2013

```How many customers placed an order and what is the average order amount?```

In [ ]:
%%sql
SELECT COUNT(DISTINCT customer_id) AS count,
       AVG(amount)                 AS avg
FROM postmates_orders

In [ ]:
df = postmates_orders
df.agg({'customer_id': 'nunique', 'amount': 'mean'})

#### ID 2014

```Which hour has the highest average order volume per day? Your output should have the hour which satisfies that condition, and average order volume.```

In [ ]:
%%sql
WITH orders_count_by_day_hour AS (SELECT EXTRACT(HOUR FROM order_timestamp_utc) AS hour,
                                         order_timestamp_utc::DATE              AS date,
                                         COUNT(id)                              AS cnt
                                  FROM postmates_orders
                                  GROUP BY hour, date),
     avg_orders_by_hour AS (SELECT hour,
                                   AVG(cnt)                             AS avg_orders,
                                   RANK() OVER (ORDER BY AVG(cnt) DESC) AS rnk
                            FROM orders_count_by_day_hour
                            GROUP BY hour)
SELECT hour, avg_orders
FROM avg_orders_by_hour
WHERE rnk = 1

In [ ]:
df = postmates_orders
df['hour'] = df['order_timestamp_utc'].dt.hour
df['day'] = df['order_timestamp_utc'].dt.date
df.groupby(['hour', 'day'], as_index=False).agg(cnt=('id', 'count')).groupby('hour', as_index=False).agg(
    avg_orders=('cnt', 'mean')).nlargest(1, 'avg_orders', keep='all')

#### ID 2015

```What cities recorded the largest growth and biggest drop in order amount between March 11, 2019, and April 11, 2019. Just compare order amounts on those two dates. Your output should include the names of the cities and the amount of growth/drop.```

In [ ]:
%%sql
WITH total_amount_by_city_in_date AS (SELECT city_id,
                                             order_timestamp_utc::DATE AS date,
                                             SUM(amount) as total_amount
                                      FROM postmates_orders
                                      WHERE order_timestamp_utc BETWEEN '2019-03-11' AND '2019-04-12'
                                      GROUP BY city_id, date),
     shifted_total_amount AS (SELECT city_id,
                                     date,
                                     LAG(total_amount) OVER ()       AS prev_total_amount,
                                     total_amount - LAG(total_amount) OVER () AS amount_diff
                              FROM total_amount_by_city_in_date)
SELECT name, amount_diff
FROM shifted_total_amount AS ta
         JOIN postmates_markets AS pm ON ta.city_id = pm.id
WHERE amount_diff =
      (SELECT MAX(amount_diff) FROM shifted_total_amount WHERE date = '2019-04-11')
   OR amount_diff =
      (SELECT MIN(amount_diff) FROM shifted_total_amount WHERE date = '2019-04-11')
ORDER BY city_id, date

In [ ]:
df = postmates_orders
df['date'] = df['order_timestamp_utc'].dt.date
total_amount_by_city_in_date = df[(df['date'] == pd.to_datetime('2019-03-11').date()) |
                                  (df['date'] == pd.to_datetime('2019-04-11').date())].groupby(['city_id', 'date'],
                                                                                               as_index=False).agg(
    total_amount=('amount', 'sum'))
total_amount_by_city_in_date['prev_total_amount'] = total_amount_by_city_in_date['total_amount'].shift(1)
total_amount_by_city_in_date['amount_diff'] = total_amount_by_city_in_date['total_amount'] - \
                                              total_amount_by_city_in_date['prev_total_amount']
result = total_amount_by_city_in_date.drop_duplicates(subset='city_id', keep='last')
max_amount_diff = result['amount_diff'].max()
min_amount_diff = result['amount_diff'].min()
result.query('amount_diff == @max_amount_diff | amount_diff == @min_amount_diff').merge(postmates_markets, how='inner',
                                                                                        left_on='city_id',
                                                                                        right_on='id')[
    ['name', 'amount_diff']]

#### ID 2016

```Which partners have ‘pizza’ in their name and are located in Boston? And what is the average order amount? Output the partner name and the average order amount.```

In [ ]:
%%sql
SELECT pp.name, AVG(po.amount) AS avg
FROM postmates_orders po
         LEFT JOIN postmates_markets pm ON po.city_id = pm.id
         LEFT JOIN postmates_partners pp ON po.seller_id = pp.id
WHERE pm.name = 'Boston'
  AND pp.name ILIKE '%pizza%'
GROUP BY pp.name

In [ ]:
df = pd.merge(pd.merge(postmates_orders, postmates_markets, how='left', left_on='city_id', right_on='id'),
              postmates_partners, how='left', left_on='seller_id', right_on='id')
df.query('name_x == "Boston" & name_y.str.contains("pizza", case=False)').groupby('name_y', as_index=False).agg(
    avg=('amount', 'mean'))

#### ID 2018

```Return a list of users with status free who didn’t make any calls in Apr 2020.```

In [ ]:
%%sql
SELECT user_id
FROM rc_users
WHERE user_id NOT IN (SELECT user_id
                      FROM rc_calls
                      WHERE EXTRACT(MONTH FROM date) = 4
                        AND EXTRACT(YEAR FROM date) = 2020)
  AND status = 'free'

In [ ]:
user_lst = rc_calls.query('date.dt.year == 2020 & date.dt.month == 4')['user_id'].to_list()
rc_users.query('~user_id.isin(@user_lst) & status == "free"')['user_id']

#### ID 2019

```Return the top 2 users in each company that called the most. Output the company_id, user_id, and the user's rank. If there are multiple users in the same rank, keep all of them.```

In [ ]:
%%sql
WITH cte AS (SELECT company_id,
                    user_id,
                    COUNT(call_id)                                              AS cnt,
                    DENSE_RANK()
                    OVER (PARTITION BY company_id ORDER BY COUNT(call_id) DESC) AS rank
             FROM rc_calls
                      JOIN rc_users USING (user_id)
             GROUP BY company_id, user_id)
SELECT company_id,
       user_id,
       rank
FROM cte
WHERE rank <= 2;

In [ ]:
df = pd.merge(rc_calls, rc_users, how='inner', on='user_id').groupby(['company_id', 'user_id'], as_index=False).agg(
    cnt=('call_id', 'count'))
df['rnk'] = df.groupby('company_id')['cnt'].rank(method='dense', ascending=False)
df.query('rnk <= 2')[['company_id', 'user_id', 'rnk']]

#### ID 2020

```Which company had the biggest month call decline from March to April 2020? Return the company_id and calls difference for the company with the highest decline.```